# Newton Method for solving nonlinear systems (or tangent method)

In previous laboratories we have seen Netwon Method for optimizing functions (i.e. gradient descent with Hessian preconditioning).

Now we are interested in Newton method for finding roots of nonlinear vector functions.

In [1]:
using Plots

In [159]:
module NewtonRootSolver
using LinearAlgebra:norm

"""
    newtonsolve_backtrack(x0, F, JF, alpha0, kmax, Ftol, c1, rho, btmax)

Apply the Netwon method for nonlinear systems to find the zeros of function F

# Arguments

- `x0::AbstractVector{T}`: a column vector of n elements representing the
    starting point for the optimization method

- `F::Function`: a function that, for each column vector x, returns the vector
    of values F(x)

- `JF::Function`: a function that, for each column vector x, return the
    Jacobian matrix JF(x)

- `alpha0::AbstractFloat`: a real number characterizing the step length of the
    optimization method (w.r.t. the merit function f) and the starting point of the
    backtracking strategy

- `kmax::Integer`: an integer scalar value characterizing the maximum number of
    iterations of the method

- `Ftol::AbstractFloat`: a real number characterizing the tolerance with
    respect to the norm of F in order to stop the method

- `c1::AbstractFloat`: the factor c1 for the Armijo condition that must be a
    scalar in (0, 1)

- `rho::AbstractFloat`: fixed factor, less that 1, used for reducing alpha0

- `btmax::Integer`: maximum number of steps for updating alpha during the
    backtracking strategy
"""
function newtonsolve_backtrack(
        x0::AbstractVector,
        F::Function,
        JF::Function,
        alpha0::AbstractFloat,
        kmax::Integer,
        Ftol::AbstractFloat,
        c1::AbstractFloat,
        rho::AbstractFloat,
        btmax::Integer
)
    f(x) = 0.5 * norm(F(x))^2 # merit function
    grad_f(x) = JF(x)'*F(x) # gradient of merit function
    
    xseq = []
    btseq = []

    k = 0
    xk = x0
    
    normFk = norm(F(xk))
    push!(xseq, xk)
    
    while true
        if normFk < Ftol
            break
        end
        
        if k > kmax
            break
        end
        
        # perhaps use an iterative method rather than a direct method
        delta_xk = JF(xk) \ (-F(xk))
        
        k = k + 1
        
        # De Armijo backtracking
        alpha = alpha0
        bt = 1
        while true
            if f(xk + alpha * delta_xk) <= (f(xk) + c1 * alpha * grad_f(xk)' * delta_xk)
                break
            end
            if bt > btmax
                break
            end
            
            alpha = rho * alpha
            bt = bt + 1
        end
        push!(btseq, bt)
        
        xk = xk + alpha * delta_xk
        
        normFk = norm(F(xk))
        push!(xseq, xk)
    end
    
    
    return xk, normFk, k, xseq, btseq
end

end
using Main.NewtonRootSolver

x0 = [-5.0, -5.0]
f1(x) = 2*x[1]-x[2]-exp(-x[1])
f2(x) = 2*x[2]-x[1]-exp(-x[2])
F(x) = [f1(x), f2(x)]
JF(x) = [2+exp(-x[1]) -1; -1 2+exp(-x[2])]
alpha0 = 1.
kmax = 5_000
Ftol = 1e-12
c1 = 0.0001
rho = 0.8
btmax = 50

xk, normFk, k, xseq, btseq = NewtonRootSolver.newtonsolve_backtrack(x0, F, JF, alpha0, kmax, Ftol, c1, rho, btmax)
@show xk
@show normFk
@show k
@show btseq
nothing

xk = [0.5671432904097838, 0.5671432904097838]
normFk = 1.5700924586837752e-16
k = 9
btseq = Any[1, 1, 1, 1, 1, 1, 1, 1, 1]


In [216]:
plot(
    begin
        contourf(range(-6, stop=2, length=500), range(-6, stop=2, length=500), (x1, x2) -> f1([x1, x2]), title="f₁", levels=25, c=:viridis, xlabel="x₁", ylabel="x₂")
        scatter!(map(x->x[1], xseq), map(x->x[2], xseq), label=false, c=:black, m=:+)
    end,
    begin
        contourf(range(-6, stop=2, length=500), range(-6, stop=2, length=500), (x1, x2) -> f2([x1, x2]), title="f₂", levels=25, c=:viridis, xlabel="x₁", ylabel="x₂")
        scatter!(map(x->x[1], xseq), map(x->x[2], xseq), label=false, c=:black, m=:+)
    end,
    size=(1000, 400)
)